

---

# Part 6: Network Security

## Chapter 20: Securing the Network Infrastructure

In the previous chapter, we explored the technologies that protect the network perimeter and monitor for threats: firewalls, IPS, VPNs, and NAC. These are critical components of a defense-in-depth strategy. However, they focus primarily on protecting the network *from* external threats and controlling access *to* the network.

There is another, equally important dimension to network security: securing the network **infrastructure itself**. The routers, switches, and other devices that form the network's backbone are high-value targets for attackers. If an attacker can compromise a core router, they can potentially monitor, redirect, or disrupt all traffic flowing through it. They can also use that compromised device as a launching pad for further attacks.

This chapter focuses on hardening network devices and implementing advanced Layer 2 security features. You will learn how to securely manage devices using SSH and AAA (Authentication, Authorization, and Accounting). You will explore essential switch security features like port security, DHCP snooping, and Dynamic ARP Inspection (DAI) that protect against common Layer 2 attacks. You will also be introduced to Control Plane Policing (CoPP), which protects the router's control plane from being overwhelmed by malicious traffic.

By the end of this chapter, you will understand that securing the network is not just about perimeter defenses; it is about hardening every component, from the physical ports on a switch to the management interfaces on a core router.

### 20.1 Secure Device Management (SSH vs. Telnet, HTTPS)

The first step in securing network infrastructure is to ensure that all administrative access to devices is secure. In the early days of networking, protocols like **Telnet** were commonly used to manage devices remotely. Telnet, however, transmits all data, including usernames and passwords, in **cleartext**. An attacker on the same network segment can easily capture this traffic and gain administrative credentials.

**SSH (Secure Shell)**

**SSH** is the secure replacement for Telnet. It provides a cryptographically secure channel for remote access to network devices. All communication, including authentication and subsequent commands, is encrypted.

- **How it works:** SSH uses a client-server model. The network device (router, switch) runs an SSH server. An administrator uses an SSH client (like PuTTY, or the command-line `ssh` client) to connect. The connection is established using TCP port 22.
- **Key Features:**
    - **Encryption:** All data is encrypted using strong ciphers like AES.
    - **Authentication:** Supports various authentication methods, including passwords and, more securely, public-key cryptography.
    - **Integrity:** Ensures that the data has not been tampered with during transmission.
- **Configuration Example (Cisco IOS):**
    ```
    hostname R1
    ip domain-name example.com
    crypto key generate rsa modulus 2048   ! Generates the encryption keys
    username admin secret strong-password   ! Creates a local user
    line vty 0 4
     transport input ssh                    ! Only allow SSH on VTY lines
     login local                            ! Use local database for authentication
    ```
- **Best Practice:** SSH should be the **only** remote access protocol enabled on network devices. Telnet should be disabled. Use SSH version 2 (SSHv2) whenever possible, as it is more secure than the older version 1.

**HTTPS for Web-Based Management**

Many modern network devices also offer a web-based management interface. Just as with remote CLI access, this interface must be secured.

- **HTTP vs. HTTPS:** The standard HTTP protocol transmits all data, including login credentials, in cleartext. **HTTPS (HTTP over SSL/TLS)** encrypts the entire web session, protecting it from eavesdropping.
- **How it works:** The device must have an SSL/TLS certificate installed. This certificate can be self-signed (generated by the device) or, preferably, issued by a trusted Certificate Authority (CA). When an administrator connects to the device's IP address using `https://`, their browser and the device perform a TLS handshake, establishing an encrypted session.
- **Best Practice:** Disable HTTP access entirely. Enable HTTPS and ensure a valid certificate is in place. Use strong encryption protocols (TLS 1.2 or 1.3) and disable older, vulnerable protocols like SSLv3.

### 20.2 AAA (Authentication, Authorization, and Accounting): RADIUS and TACACS+

As networks grow, managing local usernames and passwords on every single device becomes impractical and insecure. If an administrator leaves the company, their credentials must be changed on every router, switch, and firewall. **AAA** provides a centralized framework for managing access to network devices.

AAA stands for:

- **Authentication:** Who are you? Verifying the identity of a user or device attempting to access the network device.
- **Authorization:** What are you allowed to do? Determining what commands or actions an authenticated user is permitted to execute.
- **Accounting:** What did you do? Logging and tracking user actions for auditing, billing, and security analysis.

AAA is typically implemented using a client-server model, with the network device acting as the client and a central **AAA server** handling the authentication, authorization, and accounting functions.

The two main protocols used for communication between network devices and AAA servers are **RADIUS** and **TACACS+** .

**RADIUS (Remote Authentication Dial-In User Service)**

RADIUS is an open standard protocol (defined in RFC 2865 and 2866) that is widely used for network access control. It is commonly used for authenticating users connecting via VPN, wireless (802.1X), or dial-up.

- **Transport:** RADIUS typically uses UDP ports 1812 (authentication/authorization) and 1813 (accounting).
- **Encryption:** RADIUS only encrypts the password in the Access-Request packet. The rest of the packet (including the username and accounting data) is sent in cleartext.
- **Combined Auth and Auth:** RADIUS combines authentication and authorization in the same response. The Access-Accept packet contains authorization attributes (like the VLAN to assign or the IP address to issue).
- **Common Use Cases:** Network access (802.1X), VPN authentication, wireless authentication.

**TACACS+ (Terminal Access Controller Access-Control System Plus)**

TACACS+ is a Cisco proprietary protocol (though it has been widely implemented and documented) that is designed specifically for device administration. It is generally considered more suitable for managing access to network infrastructure than RADIUS.

- **Transport:** TACACS+ uses TCP port 49. TCP provides reliable delivery, which is important for command authorization.
- **Encryption:** TACACS+ encrypts the **entire body** of the packet (the entire payload), providing more comprehensive confidentiality than RADIUS.
- **Separate Auth and Auth:** TACACS+ separates the authentication and authorization processes. This allows for finer-grained control. A user can be authenticated, and then the AAA server can authorize specific commands on a per-command basis.
- **Command Authorization:** This is a key feature. TACACS+ can be configured to allow certain users to execute only a specific subset of commands (e.g., a help desk user can only run `show` commands, while a senior engineer can enter configuration mode). This enforces the principle of least privilege.
- **Common Use Cases:** Administrative access to routers, switches, and firewalls.

| Feature | RADIUS | TACACS+ |
| :--- | :--- | :--- |
| **Primary Use** | Network Access (VPN, Wireless, 802.1X) | Device Administration (Router/Switch CLI) |
| **Transport** | UDP (1812/1813) | TCP (49) |
| **Encryption** | Only password encrypted | Entire packet payload encrypted |
| **Auth & Auth** | Combined | Separate |
| **Command Authorization** | No (not designed for this) | Yes, per-command granularity |
| **Standard** | Open Standard (IETF) | Cisco Proprietary (though widely implemented) |

### 20.3 Port Security on Switches

At Layer 2, switches are vulnerable to various attacks. One of the simplest is an attacker plugging an unauthorized device into an unused wall jack. **Port security** is a feature on Cisco switches (and similar features on other vendors' switches) that helps mitigate this risk by limiting the number of MAC addresses allowed on a switch port.

**How Port Security Works:**

1.  An administrator enables port security on a specific switch interface.
2.  The administrator defines a **maximum number of secure MAC addresses** allowed on that port (default is 1).
3.  The administrator specifies what action the switch should take if a violation occurs:
    - **Protect:** Drops packets from unknown MAC addresses but does not send a notification. The violation counter is not incremented.
    - **Restrict:** Drops packets from unknown MAC addresses, sends a notification (SNMP trap or syslog message), and increments the violation counter.
    - **Shutdown:** (Default) Immediately places the port in an **errdisable** (error-disabled) state. The port is effectively shut down and must be manually re-enabled or automatically recovered after a timer. This is the most secure and most common action.

**Configuring Secure MAC Addresses:**

There are three ways to define which MAC addresses are considered "secure":

- **Static Secure MAC Addresses:** The administrator manually configures specific MAC addresses on the port. This is secure but not scalable.
- **Dynamic Secure MAC Addresses:** The switch learns MAC addresses dynamically as devices send traffic. These addresses are added to the running configuration but are lost when the switch reloads.
- **Sticky Secure MAC Addresses:** This is a hybrid approach. The switch dynamically learns MAC addresses, and then "sticks" them—converting them to static entries in the running configuration. These addresses can be saved to the startup configuration, making them persistent across reloads. Sticky learning is the most common deployment method.

**Port Security Configuration Example (Cisco IOS):**
```
interface GigabitEthernet0/1
 switchport mode access
 switchport port-security                 ! Enables port security on the interface
 switchport port-security maximum 2        ! Allows up to 2 MAC addresses
 switchport port-security violation shutdown ! Puts port in errdisable on violation
 switchport port-security mac-address sticky ! Enables sticky learning
 switchport port-security mac-address sticky 0050.7966.6801 ! (Optional) Pre-stick a specific MAC
```

**Limitations:** Port security is effective against casual unauthorized access, but it can be bypassed by sophisticated attackers who can spoof MAC addresses. It is a first line of defense, not a comprehensive solution.

### 20.4 DHCP Snooping and Dynamic ARP Inspection (DAI)

Two of the most common and dangerous Layer 2 attacks are DHCP starvation/DHCP spoofing and ARP spoofing (also known as ARP poisoning). Cisco switches offer two powerful features to mitigate these attacks: **DHCP Snooping** and **Dynamic ARP Inspection (DAI)** . These features work best when used together.

**DHCP Snooping**

DHCP snooping acts as a firewall between untrusted hosts (like user devices) and trusted DHCP servers. Its primary purpose is to prevent rogue DHCP servers from operating on the network and to build a database of legitimate DHCP leases that other security features (like DAI) can use.

**How DHCP Snooping Works:**

1.  **Trusted and Untrusted Ports:** The administrator designates certain switch ports as **trusted** and others as **untrusted**.
    - **Trusted Ports:** These are ports where legitimate DHCP servers are connected, or ports that connect to other switches (uplinks). DHCP messages received on trusted ports are allowed to pass through without inspection.
    - **Untrusted Ports:** These are ports connected to end-user devices (workstations, laptops, printers). DHCP messages received on untrusted ports are subject to strict inspection.
2.  **Message Filtering:** On untrusted ports, DHCP snooping:
    - **Blocks DHCP server messages:** It drops any DHCPOFFER, DHCPACK, or DHCPNAK messages, preventing a rogue DHCP server on an untrusted port from responding to clients.
    - **Enforces DHCP binding rules:** It ensures that DHCP messages follow the expected DORA sequence. For example, it will drop a DHCPACK that was not preceded by a DHCPDISCOVER from the same client.
3.  **DHCP Snooping Binding Table:** The switch builds and maintains a **DHCP snooping binding table**. This table contains the MAC address, IP address, lease time, and VLAN/port information for every successful DHCP lease obtained on an untrusted port. This table is a critical data source for other security features.

**Configuration Example (Cisco IOS):**
```
ip dhcp snooping                ! Enables DHCP snooping globally
ip dhcp snooping vlan 10,20     ! Enables DHCP snooping on specific VLANs

interface GigabitEthernet0/1
 description Uplink to Core Switch
 ip dhcp snooping trust          ! Designates this port as trusted (for uplinks/DHCP servers)

interface GigabitEthernet0/2
 description User Port
 ip dhcp snooping limit rate 10  ! (Optional) Limits DHCP packets to 10 per second to prevent DHCP starvation attacks
```

**Dynamic ARP Inspection (DAI)**

DAI is a security feature that validates ARP packets in a network. It protects against ARP spoofing (poisoning) attacks, where an attacker sends forged ARP messages to associate their MAC address with the IP address of a legitimate device (like the default gateway). DAI relies on the DHCP snooping binding table as its source of truth.

**How DAI Works:**

1.  **Trusted and Untrusted Ports:** Similar to DHCP snooping, DAI uses the concept of trusted and untrusted ports. ARP packets received on trusted ports are not inspected. ARP packets received on untrusted ports are subject to validation.
2.  **Packet Validation:** When a switch receives an ARP packet on an untrusted port, DAI intercepts it and checks its validity:
    - **IP-to-MAC Binding Validation:** It checks the sender's MAC and IP address in the ARP packet against the DHCP snooping binding table. If the MAC-to-IP mapping in the ARP packet does not match a valid entry in the binding table (or against a statically configured ARP ACL), the packet is considered invalid.
    - **Invalid ARP packets are dropped** and can be logged.
3.  **ARP ACLs:** For devices with static IP addresses (not obtained via DHCP), you can configure ARP ACLs to provide valid IP-to-MAC mappings that DAI can use for validation.

**Configuration Example (Cisco IOS):**
```
ip arp inspection vlan 10,20      ! Enables DAI on specific VLANs

interface GigabitEthernet0/1
 description Uplink to Core Switch
 ip arp inspection trust          ! Designates this port as trusted (uplinks are trusted)

interface GigabitEthernet0/2
 description User Port
 ip arp inspection limit rate 15  ! (Optional) Limits ARP packet rate to prevent ARP flooding attacks
```

By implementing DHCP snooping and DAI together, you create a powerful defense against a whole class of common Layer 2 attacks. DHCP snooping prevents rogue DHCP servers and builds the binding table; DAI uses that table to validate ARP traffic and prevent ARP spoofing.

### 20.5 Control Plane Policing (CoPP)

The final infrastructure security feature we will explore is **Control Plane Policing (CoPP)** . To understand CoPP, you must first understand the different planes of a router or switch:

- **Data Plane (Forwarding Plane):** Handles the actual user traffic that is being forwarded through the device (e.g., a packet going from a user to the Internet). This is the high-speed path.
- **Control Plane:** Runs the routing protocols (OSPF, BGP), processes ARP, handles management traffic (SSH, SNMP), and performs other functions necessary to build and maintain the network. The control plane is responsible for determining how traffic should be forwarded. It is processed by the device's CPU.
- **Management Plane:** A subset of the control plane, specifically for administrative access (SSH, HTTPS, SNMP).

The control plane is vulnerable to attack. An attacker could flood a router with traffic destined for the control plane (e.g., a massive number of SSH connection attempts, or a flood of BGP packets). This would overwhelm the CPU, preventing it from performing its normal functions (like routing updates or responding to legitimate management traffic). This is, in effect, a denial-of-service (DoS) attack against the router itself.

**CoPP** is a Cisco feature (similar features exist on other vendors' platforms) that protects the control plane. It uses standard access control lists and policing mechanisms to rate-limit the traffic that is punted to the control plane.

**How CoPP Works:**

1.  **Classify Traffic:** The administrator creates class maps to classify different types of traffic destined for the control plane. For example:
    - **Critical Traffic:** Routing protocol traffic (OSPF, BGP), which is essential for network operation. This should be allowed with a guaranteed rate.
    - **Normal Traffic:** Management traffic (SSH, SNMP), which is important but not as critical as routing protocols. This can be allowed but policed to a reasonable rate.
    - **Undesirable Traffic:** Traffic that should never reach the control plane, like Telnet (if disabled) or packets from known malicious sources. This can be dropped outright.
    - **Default Traffic:** All other traffic not explicitly classified. This should be policed to a very low rate to prevent it from overwhelming the CPU.
2.  **Create a Policy Map:** The administrator creates a policy map that applies specific actions (e.g., `transmit`, `drop`, `police`) to each class of traffic.
3.  **Apply the Policy to the Control Plane:** The policy is applied to the control plane itself using the `control-plane` configuration command. The device's CPU is then protected by the policy; any traffic exceeding the defined rates is dropped before it can impact the CPU.

**CoPP Configuration Example (Conceptual - Cisco IOS):**
```
! Classify critical routing protocols
class-map match-all CRITICAL
 match protocol ospf
 match protocol bgp

! Classify management traffic
class-map match-all MANAGEMENT
 match protocol ssh
 match protocol snmp

! Define the policy
policy-map CONTROL-PLANE-POLICY
 class CRITICAL
  police 100000 conform transmit exceed transmit  ! Allow all critical traffic, but police at a high rate
 class MANAGEMENT
  police 50000 conform transmit exceed drop       ! Police management traffic, drop excess
 class class-default
  police 10000 conform drop exceed drop            ! Police all other traffic, drop if any

! Apply the policy to the control plane
control-plane
 service-policy input CONTROL-PLANE-POLICY
```

CoPP is an advanced security feature that requires careful planning and testing. Misconfiguration can inadvertently block legitimate control plane traffic and disrupt the network. However, when correctly implemented, it is an essential tool for protecting the network's core.

---

### Chapter 20: Hands-On Challenge

Securing infrastructure requires access to network devices. Here are some practical exercises you can perform in a lab environment.

1.  **Configure SSH on a Router (in Packet Tracer or GNS3):**
    - Set up a simple topology with a router and a PC connected to it via a switch.
    - On the router, configure a hostname, domain name, generate RSA keys, create a local user, and configure the VTY lines to only accept SSH connections.
    - From the PC, use an SSH client (like the one built into Packet Tracer's PC) to connect to the router's IP address. Verify that you can log in and that Telnet is no longer accessible.

2.  **Configure a Simple AAA Login Authentication (in a lab):**
    - Extend the previous lab. On the router, configure it to point to a RADIUS or TACACS+ server. (You can simulate a server in Packet Tracer or use a free tool like FreeRADIUS on a Linux VM).
    - Configure AAA on the router to use the server for login authentication, with a fallback to the local database if the server is unreachable.
    - Test logging in with credentials from the AAA server and with local credentials.

3.  **Configure Port Security (in Packet Tracer):**
    - Connect a PC to a switch port.
    - On the switch, enable port security on that port, set the maximum MAC addresses to 1, set the violation mode to `shutdown`, and enable sticky MAC learning.
    - Test by disconnecting the PC and connecting a different device (or simulating a different MAC address). The port should go into errdisable state.
    - Re-enable the port with `shutdown` followed by `no shutdown` on the interface.

4.  **Configure DHCP Snooping and DAI (in Packet Tracer):**
    - This is a more advanced lab, but it's excellent for understanding how these features work together.
    - Set up a topology with a switch, a DHCP server, and two client PCs.
    - Configure DHCP snooping on the switch, designating the port connected to the DHCP server as `trust` and the client ports as `untrust`.
    - Configure DAI on the same VLAN.
    - Observe how the DHCP snooping binding table is built (`show ip dhcp snooping binding`).
    - Then, simulate an ARP spoofing attack (by configuring a static ARP entry on a malicious PC with the gateway's IP and a fake MAC). DAI should detect and drop the invalid ARP packets.

5.  **Research CoPP:**
    - Since CoPP is difficult to simulate in basic labs, read Cisco's documentation on CoPP. Search for "Cisco CoPP Best Practices." Look at example configurations and think about how you would classify traffic in your own network design.

---

This chapter has focused on hardening the network infrastructure itself. You now understand the importance of secure management protocols (SSH, HTTPS), the power of centralized AAA for access control, and the critical Layer 2 security features (port security, DHCP snooping, DAI) that protect against common internal attacks. You have also been introduced to CoPP, which safeguards the device's control plane.

In the next part of the book, we will look toward the future. **Part 7: Advanced Topics and Modern Networking** will explore cutting-edge technologies like network virtualization, software-defined networking (SDN), automation, advanced wireless, and structured troubleshooting methodologies.